In [1]:
import csv
import requests
import pandas as pd 
from IPython.display import clear_output

import warnings
warnings.filterwarnings('ignore')


# Censo de locales de Madrid

Utilizamos el csv obtenido del Ayuntamiento de Madrid con el censo de los locales de Madrid. Comenzamos leyendo el fichero para su tratamiento.

In [2]:
df_total = pd.read_csv('datos/raw/datosCensoLocalesActividades.csv',sep=';')

Se realiza un tratamiento del ID del barrio puesto que el formato que utiliza es distinto al que se dispone en el resto de ficheros. Es por esto que es necesario hacer un pequeño procesamiento del campo "id_barrio_local".

In [10]:
df_total['id_barrio_local'] = df_total['id_barrio_local'].str.replace('0','')
df_total

,id_local,id_distrito_local,desc_distrito_local,id_barrio_local,desc_barrio_local,cod_barrio_local,id_seccion_censal_local,desc_seccion_censal_local,coordenada_x_local,coordenada_y_local,...,desc_division,id_epigrafe,desc_epigrafe,ANIO,ï»¿id_local,coordenada_y_agrupacion,fx_carga,fx_datos_ini,fx_datos_fin,id_vial_local
0,270551186.0,6,TETUAN,61,BELLAS VISTAS,NaN,NaN,19,"440300,6","4477924,54",...,NaN,NaN,NaN,2018,NaN,NaN,NaN,NaN,NaN,NaN
1,270551189.0,1,CENTRO,14,JUSTICIA,NaN,NaN,82,0,0,...,EDUCACI0N,81,ESCUELAS INFANTILES DE PRIMER CICLO,2018,NaN,NaN,NaN,NaN,NaN,NaN
2,270551190.0,1,CENTRO,14,JUSTICIA,NaN,NaN,82,0,0,...,ALMACENAMIENTO Y ACTIVIDADES ANEXAS AL TRANSPORTE,52,APARCAMIENTOS PUBLICOS,2018,NaN,NaN,NaN,NaN,NaN,NaN
3,270551191.0,1,CENTRO,14,JUSTICIA,NaN,NaN,82,0,0,...,"ACTIVIDADES DEPORTIVAS, RECREATIVAS Y DE ENTRE...",93,PISCINAS DE USO PUBLLICO CLIMATIZADAS,2018,NaN,NaN,NaN,NaN,NaN,NaN
4,270551231.0,3,RETIRO,31,PACIFICO,NaN,NaN,1,"442392,58","4472589,51",...,ACTIVIDADES SANITARIAS,81,CONSULTA DE MEDICINA GENERAL Y ESPECIALIDADES,2018,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1605632,290001641.0,18,VILLA DE VALLECAS,181,CASCO H.VALLECAS,NaN,NaN,46,"443802,18","446831,75",...,ACTIVIDADES JURIDICAS Y DE CONTABILIDAD,NaN,"ACTIVIDADES DE CONTABILIDAD, TENEDURIA, AUDITO...",2016,NaN,NaN,NaN,NaN,NaN,NaN
1605633,290001647.0,18,VILLA DE VALLECAS,181,CASCO H.VALLECAS,NaN,NaN,46,"443919,9","446810,95",...,"SERVICIOS FINANCIEROS, EXCEPTO SEGUROS Y FONDO...",NaN,"INTERMEDIACION MONETARIA: BANCOS, CAJAS DE AHORRO",2016,NaN,NaN,NaN,NaN,NaN,NaN
1605634,290001653.0,18,VILLA DE VALLECAS,181,CASCO H.VALLECAS,NaN,NaN,46,"443948,87","446811,85",...,REPARACI0N E INSTALACI0N DE MAQUINARIA Y EQUIPO,NaN,INSTALACION DE MAQUINAS Y EQUIPOS INDUSTRIALES,2016,NaN,NaN,NaN,NaN,NaN,NaN
1605635,290001654.0,18,VILLA DE VALLECAS,181,CASCO H.VALLECAS,NaN,NaN,46,"443945,59","446812,25",...,"SERVICIOS FINANCIEROS, EXCEPTO SEGUROS Y FONDO...",NaN,"INTERMEDIACION MONETARIA: BANCOS, CAJAS DE AHORRO",2016,NaN,NaN,NaN,NaN,NaN,NaN


Obtenemos el número de locales que hay por cada barrio. Para ello hacemos un group by por el id de cada barrio y el año y sumamos el numero de locales. Posteroprmente renombramos las columnas obtenidas y almacenamos el resultado en un csv.

In [11]:
df_total = df_total[['id_distrito_local','desc_distrito_local', 'id_barrio_local','cod_barrio_local', 'desc_barrio_local','ANIO', 'id_epigrafe', 'desc_epigrafe', 'desc_division' ]]
df_final = df_total.groupby(['id_barrio_local','ANIO'], as_index=False)['desc_barrio_local'].count()
df_final.rename(columns={"desc_barrio_local":"numero_locales", "id_barrio_local":"CodigoBarrio", "ANIO":"Anio"}, inplace=True)
df_final.to_csv('datos/raw/datosCensoLocalesActividadesNumero.csv',sep=';',index=False) 
df_final

,CodigoBarrio,Anio,numero_locales
0,,2018,1
1,11,2014,2767
2,11,2015,2795
3,11,2016,2858
4,11,2017,5567
...,...,...,...
1047,97,2018,766
1048,97,2019,766
1049,97,2020,766
1050,97,2021,770


Realizamos un proceso similar al anterior para obtener el número de alojamientos de cada barrio. En este caso se debe filtrar del total del censo, aquellos locales cuyo epigrafe se encuentra entre “HOSTELERIA”, “SERVICIOS DE ALOJAMIENTO” Y “VIVIENDAS TURÍSTICAS”, que son los que se registran como alojamientos turisticos.

In [12]:
#Buscamos los epigrafes  “HOSTELERIA”, “SERVICIOS DE ALOJAMIENTO” Y “VIVIENDAS TURÍSTICAS” que se corresponden con alojamientos turisticos

epigrafes = ["HOSTELERIA", "SERVICIOS DE ALOJAMIENTO","VIVIENDAS TURÍSTICAS"]
df_alojamientos = df_total[df_total['desc_division'].isin(epigrafes)]
df_alojamientos = df_alojamientos.groupby(['id_barrio_local','ANIO'], as_index=False)['desc_epigrafe'].count()
df_alojamientos.rename(columns={"desc_epigrafe":"numero_alojamientos", "id_barrio_local":"CodigoBarrio","ANIO":"Anio"}, inplace=True)
df_alojamientos.to_csv('datos/raw/datosCensoLocalesAlojamientoNumero.csv',sep=';',index=False) 

df_alojamientos

,CodigoBarrio,Anio,numero_alojamientos
0,11,2015,24
1,11,2016,29
2,11,2017,29
3,11,2018,31
4,11,2019,34
...,...,...,...
697,97,2018,2
698,97,2019,2
699,97,2020,2
700,97,2021,3
